<center>
    <a href="https://github.com/enterprisium/SocialGPT">
        <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Ei-sc-github.svg/768px-Ei-sc-github.svg.png' height="80" alt="SocialGPT Github"/>
    </a>
</center>

In [ ]:
#@markdown <center><h2>Setup</h2>install all the required dependicies</h3></center><br>

from IPython.display import HTML
from base64 import b64encode

def play(f):
  mp4 = open(f,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

!apt install imagemagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

!wget -qO- 'http://keyserver.ubuntu.com/pks/lookup?op=get&search=0x6888550b2fc77d09' | sudo tee /etc/apt/trusted.gpg.d/songrec.asc
!sudo apt-add-repository ppa:marin-m/songrec -y -u

!sudo apt install songrec -y
!pip install youtube-transcript-api langchain langchain_openai openai requests \
moviepy==2.0.0.dev2 imageio==2.25.1 pysrt==1.1.2 Pillow==9.5.0 ffmpeg-python \
pytube google-api-python-client google-auth-oauthlib google-auth-httplib2 \
oauth2client git+https://github.com/m1guelpf/auto-subtitle.git

In [ ]:
#@markdown <center><h2>SETUP API KEYS AND DOWNLOAD LINK</h2>

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["PEXELS_API_KEY"] = userdata.get('PEXELS_API_KEY')
os.environ["YOUTUBE_SHORT_URL"] = "https://www.youtube.com/shorts/k_7meFS5Uqo"

In [ ]:
#@markdown <center><h2>Create New Folder If Not Exist</h2>

import os

directory_path = "output"

if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")
else:
    print(f"Directory '{directory_path}' already exists.")


Directory 'output' already exists.


In [ ]:
#@markdown <center><h2>Download The Short</h2>

!mkdir "tmp"
!pytube "$YOUTUBE_SHORT_URL" --audio --target "tmp"
!cd "tmp" && for fl in *.mp4; do mv "$fl" "../output/short.mp4"; done
!rm -rf "tmp"

Loading video...
WORK HARD ON YOURSELF - Motivational Speech.mp4 | 0 MB
 ↳ |███████████████████████████████████████████████████████| 100.0%


In [ ]:
#@markdown <center><h2>Extract Audio</h2>

play("output/short.mp4")

In [ ]:
 #@markdown <center><h2>Get the transcript for a video</h2>

from youtube_transcript_api import YouTubeTranscriptApi

youtube_short_url = os.getenv("YOUTUBE_SHORT_URL")
video_id = youtube_short_url.split("/")[-1]

res = YouTubeTranscriptApi.get_transcript(video_id)
transcript = " ".join(map(lambda x: x.get("text", ""), res))
transcript

"everything changed for me when I understood this philosophy learn to work harder on yourself than you do on your job that's the simple philosophy that changed my life forever starting age 25 if you work hard on your job you can make a living which is fine if you work hard on yourself you can make a fortune which is super fine you can have more than you've got because you can become more than you are"

In [ ]:
#@markdown <center><h2>Generate similar scripts</h2>
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = """Generate a vibrant simple to understand, higly motivitonal and lively script. I'll share a reference transcript,
and I want you to create a script with a similar vibe but infused with modern language to resonate with the young internet-savvy audience.
Use simple words and terminalogy and yet keep it engaging.
Additionally, provide a short and captivating video caption for the script, concluding it with a relevant emoji.
At the end of each video always ask audiounce to drop 100 in comment if they listened so far.
Reference Transcript: {transcript}
Ouput:
The caption and script must be seprated by a new line.
Do not include the word Caption: and Script: in you answer just seprate them by one new line.
Do not use new lines and emojies in script.
example:

my caption

my script
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

output = chain.invoke({"num_scripts": 1, "transcript": transcript})
scripts = output.strip().split("\n\n")
scripts

['Unlock Your Potential with Self Growth 💪',
 'The game-changer for me? Realizing this mantra: "Invest more in yourself than in your job". This single mindset switch changed my life at 25. Working hard on your job? Sure, that\'ll earn you a living - and that\'s cool. But working hard on yourself? Now that\'s where the magic happens, that\'s where you make a fortune. And that\'s not just cool, that\'s epic! Remember, you can always have more, because you can always be more. So, if you\'re still with me, drop a "100" in the comments. Let\'s grow together! 💯']

In [ ]:
#@markdown <center><h2>(Optional) Use trending background music</h2>
caption, script = scripts
caption += " #motivation #mindset #entrepreneur"
os.environ["CAPTION"] = caption
caption, script

('Unlock Your Potential with Self Growth 💪 #motivation #mindset #entrepreneur',
 'The game-changer for me? Realizing this mantra: "Invest more in yourself than in your job". This single mindset switch changed my life at 25. Working hard on your job? Sure, that\'ll earn you a living - and that\'s cool. But working hard on yourself? Now that\'s where the magic happens, that\'s where you make a fortune. And that\'s not just cool, that\'s epic! Remember, you can always have more, because you can always be more. So, if you\'re still with me, drop a "100" in the comments. Let\'s grow together! 💯')

In [ ]:
# [X] Generate a voice over for the video
#@markdown <center><h2>Generate a voice over for the video</h2>

from openai import OpenAI

client = OpenAI()

voice_path = "output/voice.mp3"
response = client.audio.speech.create(
    model="tts-1-hd",
    voice="onyx",
    input=script,
)
response.stream_to_file(voice_path)
voice_path

'output/voice.mp3'

In [ ]:

#@markdown <center><h2>Get the Audio Length </h2>

from moviepy.editor import AudioFileClip

audio = AudioFileClip(voice_path)
duration_in_seconds = round(audio.duration)
duration_in_seconds

29

In [ ]:

#@markdown <center><h2>Guess the music using shazam or similar service and get the youtube link</h2>

import json

!songrec recognize -j "output/short.mp4" > "output/shazam.json"
"output/shazam.json"

Error: No match for this song


'output/shazam.json'

In [ ]:
#@markdown <center><h2>Voice Modeficication</h2>

import os
import json
import requests
from moviepy.editor import AudioFileClip
from pytube import Search

music_title = None
try:
  shazam = json.loads(open("output/shazam.json").read())
  music_title = shazam.get("track").get("title")
except:
  pass

search_title = f"{music_title} music only" if music_title else "Ravens Rock - Sweet Dreams (Extended) [slowed and reverb]"
s = Search(search_title)
os.environ["YOUTUBE_MUSIC_URL"] = f"https://www.youtube.com/watch?v={s.results[1].video_id}"

!mkdir "tmp"
!pytube "$YOUTUBE_MUSIC_URL" --audio --target "tmp"
!cd "tmp" && for fl in *.mp4; do mv "$fl" "../output/music.mp3"; done
!rm -rf "tmp"

Loading video...
𝑺𝒘𝒆𝒆𝒕 𝑫𝒓𝒆𝒂𝒎𝒔  𝑹𝒂𝒗𝒆𝒏𝒔 𝑹𝒐𝒄𝒌 𝑺𝒍𝒐𝒘𝒆𝒅 𝒕𝒐 𝒉𝒆𝒍𝒍 (longer version).mp4 | 4 MB
 ↳ |███████████████████████████████████████████████████████| 100.0%


In [ ]:
play("output/music.mp3")

In [ ]:
#@markdown <center><h2>Generate a caption for video</h2>

!auto_subtitle "output/voice.mp3" --srt_only "True" --output_dir "output" --language "en"

import pysrt
import string

subs = pysrt.open('output/voice.srt')

for sub in subs:
    sub.text = sub.text.translate(str.maketrans("", "", string.punctuation))
    if len(sub.text) > 20:
        words = sub.text.split()
        lines = []
        current_line = ""

        for word in words:
            if len(current_line) + len(word) <= 30:
                current_line += word.title() + " "
            else:
                lines.append(current_line.strip())
                current_line = word.title() + " "

        if current_line:
            lines.append(current_line.strip())

        sub.text = "\n".join(lines)
    else:
        sub.text = sub.text.title()

subs.save("output/voice.srt")

Extracting audio from voice...
Generating subtitles for voice... This might take a while.
100% 3172/3172 [00:37<00:00, 84.14frames/s]


In [ ]:
#@markdown <center><h2>Genrate Search Terms</h2>

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

template = """Imagine you're a highly imaginative artist with the unique ability to map the subjects
in a given SRT caption to a one word real-world objects and scenes.
It's important to keep the titles exactly one word, and title must be a real word object or scences
That human can vision, avoid providing teoritocal titles like: Paradox, Chrysalis, Journey, Reality, Accomplishment
Instead use real world titles that can bee seen by human eyes like Mansion, Yoga, Car, Money.
Give me exactly {num_clips} distinct clip titles.
Each title should seamlessly flow into the next, creating a captivating narrative,
and each title will be precisely 5 seconds long.
I want you to understand and imagine the big picture of the video and give me titles that matches
The entire video, not just invidual scences.

Get inspired by the SRT caption provided:

{srt_caption}

Output Instruction:
Provide only the titles. Each title must be separated by a new line,
do not mention numbers in titles and titles must be URL-encoded friendly.
Example Output:
Yoga
Forest
Office
Jogging
Sunset
Cafe
Hiking
Spa
Beach
Tea
Luxury
Money
"""


prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

num_clips = (duration_in_seconds // 5) + 1

with open("output/voice.srt", "r") as f:
  srt_caption = f.read()
  output = chain.invoke({"num_clips": num_clips, "srt_caption": srt_caption})

clips_titles = output.strip().split("\n")
clips_titles

['Book', 'Meditation', 'Office', 'Gym', 'Treasure', 'Community']

In [ ]:


#@markdown <center><h2>download relevent videos from pexels</h2>


from collections import Counter
import os
import random
from moviepy.editor import AudioFileClip
import requests

clip_counter = Counter(clips_titles)
clips_paths = []
selected_videos = set()

for title, count in clip_counter.items():
    headers = {
        "Authorization": os.environ["PEXELS_API_KEY"],
    }
    url = f"https://api.pexels.com/videos/search?query={title}&per_page=15&orientation=portrait&size=medium"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        videos = response.json()["videos"]
        for i in range(count):
            available_videos = [
                video
                for video in videos
                if video["video_files"][0]["link"] not in selected_videos
            ]
            if not available_videos:
                print(f"No more available videos for title '{title}'.")
                break

            video = random.choice(available_videos)
            video_url = video["video_files"][0]["link"]
            temp_name = f"{title}_{i}.mp4"
            video_path = os.path.join("output", temp_name)
            with open(video_path, "wb") as video_file:
                video_file.write(requests.get(video_url).content)
            clips_paths.append(video_path)
            selected_videos.add(video_url)
    else:
        print(
            f"Failed to fetch videos for title '{title}'. Status code: {response.status_code}"
        )

for i in range(num_clips - len(clips_paths)):
  clips_paths.append(clips_paths[0])

clips_paths

['output/Book_0.mp4',
 'output/Meditation_0.mp4',
 'output/Office_0.mp4',
 'output/Gym_0.mp4',
 'output/Treasure_0.mp4',
 'output/Community_0.mp4']

In [ ]:
#@markdown <center><h2>Adjust Size length and ratio of the downloaded clips with Moviepy
</h2>


from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.fx.all import resize


def resize_clip(input_video_path, duration=5, new_dimensions = (720, 1280)):
    video_clip = VideoFileClip(input_video_path)
    total_duration = video_clip.duration
    start_time = (total_duration - duration) / 2
    end_time = start_time + duration
    video_clip = video_clip.subclip(start_time, end_time)

    video_clip = resize(video_clip, new_dimensions)
    return video_clip


clips = [resize_clip(cp) for cp in clips_paths]
clips

In [ ]:
#@markdown <center><h2>Combine All in 1 Video VoiceOver, Music, Video Clips</h2>

from moviepy.editor import VideoFileClip, CompositeAudioClip, concatenate_videoclips
from moviepy.audio.fx.all import volumex

voice_audio = AudioFileClip("output/voice.mp3").fx(volumex, 3)
music_audio = AudioFileClip("output/music.mp3").fx(volumex, 0.5)
t_start = int(music_audio.duration // 2 - duration_in_seconds // 2)
music_audio = music_audio.subclip(t_start, duration_in_seconds + t_start)
final_clip = concatenate_videoclips(clips, method="compose").subclip(0, duration_in_seconds)
audio = CompositeAudioClip([voice_audio, music_audio])
final_clip = final_clip.set_audio(audio)

final_clip.write_videofile("output/composite_video.mp4", codec="libx264",
                           audio_codec="aac", fps=24)

Moviepy - Building video output/composite_video.mp4.
MoviePy - Writing audio in composite_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output/composite_video.mp4



Moviepy - Done !
Moviepy - video ready output/composite_video.mp4


In [ ]:
#@markdown <center><h2>final Video</h2>

play("output/composite_video.mp4")

In [ ]:
#@markdown <center><h2>Add Subtitles And WaterMark In the Final Video</h2>

from moviepy.editor import TextClip, CompositeVideoClip
from moviepy.video.tools.subtitles import SubtitlesClip

video = VideoFileClip("output/composite_video.mp4")
watermark_clip = TextClip("@MojoVibesDaily", font="Nimbus-Sans-Bold", fontsize=24,
                          color='white', size=(640, 480)).set_duration(video.duration)
generator = lambda txt: TextClip(txt, font="Nimbus-Sans-Bold", fontsize=36,
                                 color='white', size=(video.w, video.h),
                                 stroke_color='black', stroke_width=1.5)
subtitles = SubtitlesClip("output/voice.srt", generator)
result = CompositeVideoClip([video, subtitles.set_position(('center')),
                             watermark_clip.set_position(('center', 'bottom'))])
result.write_videofile("output/out.mp4", fps=video.fps, codec="libx264",
                       audio_codec="aac")

Moviepy - Building video output/out.mp4.
MoviePy - Writing audio in outTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output/out.mp4



t:  91%|█████████▏| 697/762 [01:59<00:12,  5.02it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file output/composite_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame index 696 (out of a total 762 frames), at time 29.00/31.75 sec. Using the last valid frame instead.
  warnings.warn(
t:  92%|█████████▏| 698/762 [01:59<00:11,  5.57it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file output/composite_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame index 697 (out of a total 762 frames), at time 29.04/31.75 sec. Using the last valid frame instead.
  warnings.warn(
t:  92%|█████████▏| 699/762 [01:59<00:12,  5.22it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file output/composite_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame index 698 (out of a total 762 frames), at time 29.08/31

Moviepy - Done !
Moviepy - video ready output/out.mp4


In [ ]:
#@markdown <center><h2>Get The Final Video</h2>

print(caption)
play("output/out.mp4")

In [ ]:
#@markdown <center><h2>Upload The Final Video To Youtube</h2>

from google.colab import userdata

with open("client_secrets.json", "w") as f:
  f.write(userdata.get('CLIENT_SECRETS'))
with open("upload_video.py-oauth2.json", "w") as f:
  f.write(userdata.get('PY_OAUTH2'))

!wget https://gist.githubusercontent.com/tajpouria/f37f4999ba4913369878b1c12be83436/raw/6ec94f4e1457a688fcda9869156cbad3af89b0c9/upload_video.py

--2024-01-14 11:54:35--  https://gist.githubusercontent.com/tajpouria/f37f4999ba4913369878b1c12be83436/raw/6ec94f4e1457a688fcda9869156cbad3af89b0c9/upload_video.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6968 (6.8K) [text/plain]
Saving to: ‘upload_video.py.2’

upload_video.py.2   100%[===================>]   6.80K  --.-KB/s    in 0s      

2024-01-14 11:54:35 (46.3 MB/s) - ‘upload_video.py.2’ saved [6968/6968]

